<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [345]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly.express as px

In [346]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432 

In [347]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [348]:
# текст запроса
query = f'''select count(id)
    from public.vacancies '''

In [349]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [350]:
# текст запроса
query = f'''select count(id)
    from public.EMPLOYERS '''

In [351]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [352]:
# текст запроса
query = f'''select count(id)
    from public.AREAS '''

In [353]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [354]:
# текст запроса
query = f'''select count(id)
    from public.INDUSTRIES '''

In [355]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

# Выводы по предварительному анализу данных
##### __В данном юните__ мы ведем общий подсчёт наименований которые есть в нашей базе данных. Таким образом можно сделать выводы, что кол-во вакансий больше в _2_  раза чем кол-во работодателей, то есть можно предпорложить, что у некоторых работодателей выставлено больше чем _одна_ вакансия. Так же вакансии охватывают _1362_ региона и _294_ сферы деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [356]:
# текст запроса
query = f'''select a.name,
    count(v.id)
    from public.VACANCIES as v
    join public.AREAS as a  on v.area_id = a.id
    group by 1
    order by 2 desc
    limit 5
     '''

In [357]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [358]:
# текст запроса
query = f'''select count(v.id)
    from public.VACANCIES as v
    where v.salary_from is not null or v.salary_to is not null 
     '''

In [359]:
# результат запроса
df = pd.read_sql_query(query,connection )
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\2792758036.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [360]:
# текст запроса
query = f'''select  ROUND (avg(v.salary_from)),
        ROUND(avg(v.salary_to))
    from public.VACANCIES as v
    '''

In [361]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [362]:
# текст запроса
query = f'''select v.schedule,
    v.employment,
    count(v.id)
    from public.VACANCIES as v
    group by 1,2
    order by 3 desc
    
     '''

In [363]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [364]:
# текст запроса
query = f'''select v.experience,
    count(v.id)
    from public.VACANCIES as v
    group by 1
    order by 2 asc
    
     '''
    

In [365]:
# результат запроса
df = pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1026051134.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [366]:
# выводы по детальному анализу вакансий
query = f'''select *,
    a.name as region
    from public.VACANCIES as v
    join public.areas as a on v.AREA_ID = a.id
    
    
     '''
    
    
df = pd.read_sql_query(query,connection)
result = df.copy()
result.drop('name',axis=1,inplace=True)
a = result.groupby(['region'],as_index=False).agg({'area_id':'count'}).sort_values('area_id',ascending=False).iloc[:5]
display(a)

res  = px.bar(data_frame= a,
              x= 'region',
              y= 'area_id',
              height=500,
              width=1000,
              color= 'region',
              template= 'presentation',
              title= 'Сколько вакансий в каждом регионе')
res.update_layout(xaxis_title = 'РЕГИОН',
                  yaxis_title = 'КОЛ_ВО',
                  title = {'font':dict(size = 35),
                           'x':0.5,
                           }, showlegend = False)
res.update_traces(hovertemplate = 'Регион: %{x} <br> Кол-во: %{y}')
res.show()
res.write_html('write_html/graphic_1.html')

query = f'''select v.schedule,
    v.employment,
    count(v.id)
    from public.VACANCIES as v
    group by 1,2
    order by 3 desc
    
     '''
     
df= pd.read_sql_query(query,connection)
res = df.groupby(['schedule','employment'],as_index=False).agg({'count':'mean'}).sort_values('count',ascending=False).iloc[:5]
display(res)
gr = px.bar(data_frame=res,
            x='schedule',
            y= 'count',
            height = 500,
            width= 1000,
            color='employment',
            template='plotly_dark',
            title= 'количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства',
            barmode= 'group'
           )
gr.update_layout(xaxis_title = 'ГРАФИК',
                 yaxis_title = 'КОЛ=-ВО',
                 title = {'font': dict(size = 15),
                          'x': 0.5,})
gr.update_traces(hovertemplate =  'График %{x} <br> Кол-во %{y}')
gr.show()
gr.write_html('write_html/graphic_2.html')



C:\Users\user\AppData\Local\Temp\ipykernel_34344\293232022.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,region,area_id
409,Москва,5333
564,Санкт-Петербург,2851
395,Минск,2112
457,Новосибирск,2006
21,Алматы,1892


C:\Users\user\AppData\Local\Temp\ipykernel_34344\293232022.py:43: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,count
6,Полный день,Полная занятость,35367.0
14,Удаленная работа,Полная занятость,7802.0
2,Гибкий график,Полная занятость,1593.0
17,Удаленная работа,Частичная занятость,1312.0
10,Сменный график,Полная занятость,940.0


# Выводы по детальному анализу вакансий
 __На данных графиках видно:__ что большее кол-во вакансий находится в _Москве_ и _Санкт-Петербурге_. Так же известа желаемая среднее значение ЗП для минимальной и максимальной вилок, оно составило _71065.0_ и _110537.0 тыс.руб_.__На втором графике видно__ что максимальное кол-во вакансий приходится на сочетание _Полного рабочего дня_ и _Полной занятости_. С помощью построения SQL- запроса мы выясняли что максимум вакансий в которых указан _Требуемый опыт работы_ является опыт _Более 6 лет_, на втором месте _без опыта работы_. Из этого можно сделать вывод что на рынке труда требуются специалисты с высоким стажем труда, а второе место _без опыта работы_ можно объяснить тем ,что работодатели ищут рабочих без спец. подготовки на более низкие ЗП (и таких вакансий всегда было много). Чуть меньше требуются соискатели со стажем менее 6 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [367]:
# текст запроса
query = f'''select e.name,
    count(v.id)
    from public.VACANCIES as v
    join public.EMPLOYERS as e on e.id = v.employer_id
    group by 1
    order by 2 desc
    limit 5
    '''

In [368]:
# результат запроса
df= pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\568672358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [369]:
# текст запроса
query = f'''select a.name,
    count(e.id) as "работодатели",
    count(v.id)
    from public.AREAS as a
    left join public.EMPLOYERS as e on a.id = e.area
    left join public.VACANCIES as v on a.id = v.area_id
    where v.id is null
    group by 1
    order by 2 desc
    limit 1
    '''

In [370]:
# результат запроса
df= pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\568672358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,работодатели,count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [371]:
# текст запроса
query = f'''select e.name,
    count(distinct v.area_id)
    from public.EMPLOYERS as e 
    join public.VACANCIES as v  on e.id = v.employer_id
    group by 1
    order by 2 desc
    limit 1
    '''

In [372]:
# результат запроса
df= pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\568672358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [373]:
# текст запроса
query = f'''select count(e.id)
    from public.EMPLOYERS as e 
    left join public.EMPLOYERS_INDUSTRIES as a on e.id = a.employer_id
    where a.industry_id is null
    '''


In [374]:
# результат запроса
df= pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\568672358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [375]:
# текст запроса
query = f'''select e.name,
    count(distinct a.industry_id) 
    from  public.VACANCIES as v 
    join public.EMPLOYERS as e on v.employer_id = e.id
    join public.EMPLOYERS_INDUSTRIES as a on v.employer_id = a.employer_id
    group by e.name
    having count(distinct a.industry_id) = 4
    offset  2 limit 1
    '''

In [376]:
# результат запроса
df= pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\568672358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [377]:
# текст запроса
query = f'''select  count(a.employer_id)
    from public.EMPLOYERS_INDUSTRIES as a 
    join public.INDUSTRIES as i on a.industry_id = i.id
    where i.name = 'Разработка программного обеспечения'
    '''

In [378]:
# результат запроса
df= pd.read_sql_query(query,connection)
display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\568672358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [379]:
# код для получения списка городов-милионников
url  = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 YaBrowser/23.7.5.704 Yowser/2.5 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'}

response = requests.get(url,headers=headers)
soup  = BeautifulSoup(response.text,'lxml')
content = soup.find_all('td',style = 'text-align:center')
lst  = []
for x in content:
    res = x.previous_element.find('a').text.strip()
    lst.append(res)
print(lst)



['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [380]:
# текст запроса
query = f'''select a.name,
    count(v.id) as cnt 
    from public.VACANCIES as v 
    join public.EMPLOYERS as e on v.employer_id = e.id 
    join public.AREAS as a on v.area_id = a.id
    where e.name = 'Яндекс' and  a.name in {tuple(lst)}
    group by 1 
    union all
    select 'total',
        count(distinct v.id)
    from public.VACANCIES as v 
    join public.EMPLOYERS as e on v.employer_id = e.id 
    join public.AREAS as a on v.area_id = a.id
    where e.name = 'Яндекс' and a.name in {tuple(lst)}
    order by 2 desc
    '''

In [381]:
# результат запроса
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\192212943.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,total,485
1,Москва,54
2,Санкт-Петербург,42
3,Екатеринбург,39
4,Нижний Новгород,36
5,Новосибирск,35
6,Воронеж,32
7,Краснодар,30
8,Самара,26
9,Уфа,26


***

In [382]:
# выводы по анализу работодателей
query = f'''select e.name,
    count(v.id)
    from public.VACANCIES as v
    join public.EMPLOYERS as e on e.id = v.employer_id
    group by 1
    order by 2 desc
    limit 5
    '''
  
df_1= pd.read_sql_query(query,connection)

display(df_1)

res_1  = px.bar(data_frame=df_1,
              x = 'name',
              y = 'count',
              height=500,
              width= 1000,
              color = 'name',
              template='plotly',
              title= 'График отображающий первые 5 компаний по кол-ву вакансий')
res_1.update_layout(xaxis_title = 'Компании',
                    yaxis_title = 'Кол-во',
                    title = {'font':dict(size = 25),
                             'x': 0.5,
                             'y':0.9})
res_1.update_traces(hovertemplate = 'Компании %{x} <br> Кол-во %{y}')
res_1.write_html('write_html/graphic_3.html')
res_1.show()

query_2 = f'''select a.name,
    count(v.id),
    count(e.id)
    from public.Areas as a
    join public.VACANCIES as v on a.id = v.area_id
    join public.EMPLOYERS as e on e.id = v.employer_id 
    group by 1
    order by 2 desc
    limit 5
    '''
    
df_2= pd.read_sql_query(query_2,connection)
display(df_2)

query_4 = f'''select  count(a.employer_id),'разработка програмного обеспечения' as field_of_activity
    from public.EMPLOYERS_INDUSTRIES as a 
    join public.INDUSTRIES as i on a.industry_id = i.id
    where i.name = 'Разработка программного обеспечения'
    union 
    select  count(a.employer_id),'другие'
    from public.EMPLOYERS_INDUSTRIES as a 
    join public.INDUSTRIES as i on a.industry_id = i.id
    where i.name != 'Разработка программного обеспечения'
    
'''

df_4= pd.read_sql_query(query_4,connection)
res_4 = px.pie(data_frame=df_4,
             names= 'field_of_activity',
             values='count',
             height=500,
             width=1000,
             template='plotly_dark',
             title= 'График соотношения сфер деятельности',
             hover_data= 'field_of_activity',
             labels= {'field_of_activity':'Сфера деятельности',
                     'count':'Кол-во'})
res_4.update_layout(title = {'font':dict(size =25),
                            'x':0.5})
res_4.write_html('write_html/graphic_4.html')
res_4.show()



C:\Users\user\AppData\Local\Temp\ipykernel_34344\1896817756.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1896817756.py:43: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count,count
0,Москва,5333,5333
1,Санкт-Петербург,2851,2851
2,Минск,2112,2112
3,Новосибирск,2006,2006
4,Алматы,1892,1892


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1896817756.py:58: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# ВЫВОДЫ: 
 __По данным иследованиям в юните можно предполоржить__: что топ 5 компаний по кол-ву вакансий чаще всего размещают свои вакансии в крупных городах таких как _Москва_,_Санкт-Петербург_,_Минск_,_Новосибирск_,_Алматы_. В связи с тем, что у компании _"Яндекс"_наибольшее количество вакансий можно сказать, что они размещены в наибольшем кол-ве регионов,но чаще всего в _Москве_,_Санкт-Петербурге_,_Екатеренбурге_. Так же мы видим, в процентном соотношении число работодателей с сферой  деятельности "Разработка програмного обеспечения" составило _11%_ от общего колличества работодателей.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [383]:
# текст запроса
query = f'''select count(v.id) 
    from public.VACANCIES as v 
    where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
    '''
  

In [384]:
# результат запроса
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\192212943.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [385]:
# текст запрос
query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    
    select count(t.name)
    from table_1 as t
    where lower(t.name) like '%junior%' or lower(t.experience) like '%нет опыта%' or  lower(t.employment) like '%стажировка%'
    '''

In [386]:
# результат запроса
  
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\3432893560.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [388]:
# текст запроса
query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    
    select count(t.id)
    from table_1 as t
    where t.key_skills is not null and (t.key_skills ILIKE '%SQL%' or t.key_skills ILIKE '%postgres%')
    '''

In [389]:
# результат запроса
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\192212943.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [390]:
# текст запроса
query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    
    select count(t.id)
    from table_1 as t
    where t.key_skills is not null and t.key_skills ILIKE '%Python%'
    '''

In [391]:
# результат запроса
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\192212943.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [392]:
# текст запроса
query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    select  ROUND(AVG(length(t.key_skills)-LENGTH(replace(t.key_skills,CHR(9),'')) +1),2)
    from table_1 as t
    where t.key_skills is not NULL
    '''
  

In [393]:
# результат запроса
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\192212943.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [394]:
# текст запроса
query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    
    select  distinct t.experience,
    round(avg(coalesce((t.salary_from + t.salary_to)/2,t.salary_from,t.salary_to)))
    from table_1 as t
    where t.salary_to is not null or t.salary_from is not null
    group by 1
    '''

In [395]:
# результат запроса
df= pd.read_sql_query(query,connection)

display(df)


C:\Users\user\AppData\Local\Temp\ipykernel_34344\192212943.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,round
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

In [396]:
# выводы по предметному анализу
query_5 = f'''select v.experience , 
    count(distinct id)
    from public.VACANCIES as v 
    group by 1
    
'''
df_5  = pd.read_sql_query(query_5,connection)
res_5 = px.pie(data_frame=df_5,
             names='experience',
             values='count',
             height=500,
             width=1000,
             color= 'experience',
             template='presentation',
             title='Соотношение количества вакансий с разными "Требованиями к опыту"',
             labels={'experience': 'требования к опыту',
                     'count': 'кол-во вакансий'})
res_5.update_layout(title = {'font':dict(size = 25),
                           'x':0.5})
res_5.show()
res_5.write_html('write_html/graphic_5.html')

query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    
    select t.experience,
    count(t.id)
    from table_1 as t 
    group by 1
    order by 2 desc

'''
df = pd.read_sql_query(query,connection)
display(df)

query = f'''  with table_1 as (select *
    from public.VACANCIES as v 
    where (lower(v.name)  like  '%data_science%')
    or (lower(v.name) like '%data_scientist%' )
    or (lower(v.name) like 'исследователь_данных%') 
    or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') 
    or (lower(v.name) like '%machine_learning%') 
    or (lower(v.name) like '%машинн%обучен%'))
    
    select  a.name,
        round(avg(coalesce(t.salary_from,salary_to,(t.salary_from + t.salary_to)/2)))
    from table_1 as t
    join public.areas as a on t.area_id = a.id
    where t.salary_from is not null or t.salary_to is not null
    group by 1
    order by 2 desc
    limit 10 
    '''
df_6  = pd.read_sql_query(query,connection)
display(df_6)

res_6  = px.pie(data_frame=df_6,
              names='name',
              values='round',
              height=500,
              width= 1000,
              color='name',
              template='gridon',
              labels= {'name':'Регион',
                       'round':'Средняя ЗП'},
              title= 'Соотношение средней ЗП к региону')
res_6.update_layout(title = {'font':dict(size= 35),
                           'x': 0.45})
res_6.show()
res_6.write_html('write_html/graphic_6.html')


C:\Users\user\AppData\Local\Temp\ipykernel_34344\3737956966.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



C:\Users\user\AppData\Local\Temp\ipykernel_34344\3737956966.py:40: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,count
0,От 1 года до 3 лет,225
1,От 3 до 6 лет,195
2,Более 6 лет,34
3,Нет опыта,28


C:\Users\user\AppData\Local\Temp\ipykernel_34344\3737956966.py:61: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,round
0,Кипр,300000.0
1,Армения,268863.0
2,Черногория,233794.0
3,Турция,233794.0
4,Сербия,233794.0
5,Белгород,200000.0
6,Москва,169259.0
7,Новосибирск,159667.0
8,Санкт-Петербург,154049.0
9,Королев,130000.0


# ВЫВОДЫ:
__Вакансии имеющие отношение к данным__ имеет малый процент от общего кол-ва вакансий и составило _всего 1771_ вакансию. Можно сказать что на рынке друда неохотно берут специалистов по DS без опыта, тем неменее такие вакансии есть и их кол-во составило _51 вакансию_. На графике видно, что в общей массе более 50 процентов требуются специалисты от 1 до 3, примерно такую же картину мы видим к требованиям именно для дата-сайнтистов. Так же мы видим, что уровень средней ЗП платы для DS специалистов выше за рубежом нежели в России, можно предположить, что некоторые специалисты мигрируют зарубеж ради более высоких ЗП, что в свою очередь вызовет дефицит DS специалистов в наших регионах страны.

# Общий вывод по проекту

* подведем итог исследования, обобщите выводы
* здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследован
__Проведя анализ датасета мы можем сделать общий вывод__: что вакансии и работодатели которые выставляют эти вакансии охватывают регионы _России и за её пределами_.Так же можем сделать вывод, что ЗП в среднем выше за рубежом нежели в регионах России, но кол-во вакансий намного больше в наших регионах и лидеами являются _Москва и Санкт-Петербург_, а лидером среди компаний по кол-ву вакансий является _Яндекс_. Вакансии имеющие отношение к данным составило _11%_ от общего кол-ва вакансий. Так же можно сказать о дастоточно популярном яз. програмирования как "Python", много вакансий которые требуют знания _Python_ и их _3461 вакансия_ от общего числа.

In [397]:
query = f''' 
    select  a.name,
        round(avg(coalesce(v.salary_from,salary_to,(v.salary_from + v.salary_to)/2)))
    from public.VACANCIES as v 
    join public.areas as a on v.area_id = a.id
    where v.salary_from is not null or v.salary_to is not null
    group by 1
    order by 2 desc
    limit 10 
    '''
df_6  = pd.read_sql_query(query,connection )
display(df_6)


query = f'''
    select count(v.id) , 'Python' as "навыки"
    from  public.VACANCIES as v
    where v.key_skills is not null and v.key_skills ILIKE '%Python%'
    union all
    select count(v.id) , 'другие'
    from public.VACANCIES as v
    where v.key_skills is not null and v.key_skills  not ILIKE '%Python%'
    '''


df  = pd.read_sql_query(query,connection)
display(df)
connection.close()

C:\Users\user\AppData\Local\Temp\ipykernel_34344\1143733291.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,round
0,Германия,348270.0
1,Болгария,307681.0
2,Испания,300000.0
3,Черногория,270324.0
4,Литва,263019.0
5,Польша,257156.0
6,Петропавловка (Республика Бурятия),250000.0
7,Кипр,209573.0
8,Армения,206208.0
9,Турция,204919.0


C:\Users\user\AppData\Local\Temp\ipykernel_34344\1143733291.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count,навыки
0,3461,Python
1,37280,другие
